In [9]:
using Distributions
using QuadGK

function skewness_quadgk(dist)
    μ = mean(dist)
    σ = std(dist)
    quadgk(x -> ((x-μ)/σ)^3 * pdf(dist, x), extrema(dist)...)[1]
end

function kurtosis_quadgk(dist)
    μ = mean(dist)
    σ = std(dist)
    quadgk(x -> ((x-μ)/σ)^4 * pdf(dist, x), extrema(dist)...)[1] - 3
end

function meanvarstdskku(dist)
    m = mean(dist)
    s2 = var(dist)
    s = √s2
    sk = skewness(dist)
    ku = kurtosis(dist)
    m, s2, s, sk, ku
end

function meanvarstdskku_quadgk(dist)
    m = mean(dist)
    s2 = quadgk(x -> (x - m)^2 * pdf(dist, x), extrema(dist)...)[1]
    s = √s2
    sk = quadgk(x -> ((x - m)/s)^3 * pdf(dist, x), extrema(dist)...)[1]
    ku = quadgk(x -> ((x - m)/s)^4 * pdf(dist, x), extrema(dist)...)[1] - 3
    m, s2, s, sk, ku
end

function meanvarstdskku(dist::MixtureModel)
    M1, M2, M3, M4 = moment1234_quadgk(dist)
    m = mean(dist)
    s2 = var(dist)
    s = √s2
    s3, s4 = s*s2, s2^2
    sk = 1/s3 * (M3 - 3m*s2 - m^3)
    ku = 1/s4 * (M4 - 4m*s3*sk - 6m^2*s2 - m^4) - 3
    m, s2, s, sk, ku
end

function moment1234(dist)
    m, s2, s, sk, ku = meanvarstdskku(dist)
    s3, s4 = s*s2, s2^2
    m1 = m
    m2 = s2 + m^2
    m3 = s3*sk + 3m*s2 + m^3
    m4 = s4*(ku + 3) + 4m*s3*sk + 6m^2*s2 + m^4
    m1, m2, m3, m4
end

function moment1234_quadgk(dist)
    m1 = quadgk(x -> x * pdf(dist, x), extrema(dist)...)[1]
    m2 = quadgk(x -> x^2 * pdf(dist, x), extrema(dist)...)[1]
    m3 = quadgk(x -> x^3 * pdf(dist, x), extrema(dist)...)[1]
    m4 = quadgk(x -> x^4 * pdf(dist, x), extrema(dist)...)[1]
    m1, m2, m3, m4
end

function moment1234(dist::MixtureModel)
    T = float(eltype(dist))
    M1, M2, M3, M4 = zero(T), zero(T), zero(T), zero(T)
    for (d, q) in zip(components(dist), probs(dist))
        m1, m2, m3, m4 = moment1234(d)
        M1 += q * m1
        M2 += q * m2
        M3 += q * m3
        M4 += q * m4
    end
    M1, M2, M3, M4
end

moment1234 (generic function with 2 methods)

In [10]:
T = float(eltype(Bernoulli(0.5)))
zero(T)

0.0

In [11]:
dist = Gamma(2, 3)
@show meanvarstdskku(dist) meanvarstdskku_quadgk(dist)
@show moment1234(dist) moment1234_quadgk(dist);

meanvarstdskku(dist) = (6.0, 18.0, 4.242640687119285, 1.414213562373095, 3.0)
meanvarstdskku_quadgk(dist) = (6.0, 18.00000000000002, 4.242640687119287, 1.4142135623730994, 2.9999999999999822)
moment1234(dist) = (6.0, 54.0, 648.0, 9720.0)
moment1234_quadgk(dist) = (5.999999999999999, 54.000000000000014, 648.0000000000008, 9719.999999999909)


In [12]:
mix = MixtureModel([Normal(), Normal(20)], [0.95, 0.05])
@show moment1234(mix)
@show moment1234_quadgk(mix)
@show meanvarstdskku(mix)
@show meanvarstdskku_quadgk(mix);

moment1234(mix) = (1.0, 21.0, 403.0, 8123.000000000001)
moment1234_quadgk(mix) = (0.9999999999999988, 20.999999999999982, 403.00000000022675, 8122.999999997206)
meanvarstdskku(mix) = (1.0, 20.0, 4.47213595499958, 3.8236762415271754, 13.58499999999075)
meanvarstdskku_quadgk(mix) = (1.0, 19.99999999999998, 4.472135954999577, 3.8236762415246552, 13.584999999991009)
